# Application Review

### 225HP @ 2200

In [ ]:
import numpy as np
from nptdms import tdms
import matplotlib.pyplot as plt

from acoustics import Signal
from acoustics.standards.iec_61672_1_2013 import (NOMINAL_OCTAVE_CENTER_FREQUENCIES,
                                                  NOMINAL_THIRD_OCTAVE_CENTER_FREQUENCIES)
import sig

%matplotlib inline

### Path to TDMS file:

In [ ]:
tdmspath = "./TDMS/vib.tdms"

### Setup Data Structures and Perform Standard calculations

In [ ]:
#create an instance of tdms data from the specified TDMS file
tdfile = tdms.TdmsFile(tdmspath)

'''
for channel in tdfile.group_channels('Vibration'):
    print(channel.properties)
    print("\n")
'''

In [ ]:
vib_channels = tdfile.group_channels('Vibration')
print(vib_channels[0].raw_data.shape)

#for channel in vib_channels:
#    print(channel.properties)

In [ ]:
data = tdfile.channel_data('Vibration', 'DOC X')
data
dataProp = tdfile.group_channels('Vibration')[0].properties
DOCSensitivity = float(dataProp['Sensor Sensitivity (mV/EU)'])
DOCSensitivity
gData = (data * 1000 / DOCSensitivity)

### Plot overall Run.  Look at index at bottom for more detailed plot in next cell.

In [ ]:
#df = tdfile.as_dataframe()
#df.plot(figsize=(12,24), color=('r'), subplots=True );

In [ ]:
#Convert Data to correct units.  
    #Apply the channel sensitivity.
N = 8192
T = 1/8533.3333
yf = scipy.fft(gData[1:8192])
w = hanning(N)
ywf = (scipy.fft(gData[:8192])*w)
xf = np.linspace(0.0, 1.0/(2.0*T), N/2)
plt.semilogy(xf[1:N//2], 2.0/N * np.abs(yf[1:N//2]), '-b')
plt.semilogy(xf[1:N//2], 2.0/N * np.abs(ywf[1:N//2]), '-r')
plt.legend(['FFT', 'FFT w. window'])
plt.grid()
plt.show()

In [ ]:
#Window The Data

plt.specgram(gData, NFFT=8192, Fs=8533.3333, Fc=None, detrend=None, window=np.hanning(8192), mode='psd', noverlap=6144, pad_to=None, scale='dB' )
plt.show()

In [ ]:
s = Signal(gData, fs=8533)

In [ ]:
s.plot_power_spectrum(reference=1.0)

In [ ]:
win = np.hanning(8192)
print(win)
sx = np.array(s)

In [ ]:
x = sig.apply_window(sx, win)
x.shape

In [ ]:
sig.octaves(x, s.fs, density=False, frequencies=NOMINAL_OCTAVE_CENTER_FREQUENCIES, ref=1.0)